#**README**



---



## About the notebook

This notebook demonstrates a Prompt Engineering technique called **Chain-Of-Thought(CoT)** when combined with **few-shot** prompting. It shows how CoT prompting enhances an AI model's ability to solve complex, multi-step problems.<br>



*Pre-requisites:* To run this notebook,

- Refer to [Google Colab](https://colab.research.google.com/) to get started instantly, for *free* !

- Download and open this notebook in Google Colab.

- Get your access key to the [OpenAI](https://platform.openai.com/account/api-keys) API.<br>

- Set up this access key under *secrets* in your Google Colab runtime environment. </br>
How to configure API access keys in Colab:

  - Go to the "🔑" icon in the left sidebar (Secrets).
  - Click "Add new secret".
  - For the name, use 'openai_api_key'.
  - For the value, paste your OpenAI API key.
  - Make sure "Notebook access" is enabled for this secret.

- You are all set! Have fun!



---



## Topic: Chain-Of-Thought + few-shots Prompting
Evolving models with sophisticated reasoning abilities thrive on techniques such as Chain-of-Thought(CoT), few-shot prompting.<br>
CoT prompting technique enables complex reasoning capabilities through intermediate reasoning steps.<br>

Here is an example of customer servicing ChatBot experience before CoT:<br>

- *customer:* "I want to change my subscription from standard to premium for 3 months."
- *chat-bot:* "Sure, I'd be happy to assist you with that. I just need to verify some information: - Can I have your full name, please? - Could you also provide me with your account or subscription number for verification? As soon as I verify this information, I'll be able to process your request to upgrade to the premium subscription. Thank you!"

This response is quick but lacks the necessary context and transparency from the customer's point of view. It does not inform the customer how the model calculated these fees and whether it's being applied for each month or as a one-time fee to upgrade the subscription.<br>

Now, here is how experience changes with CoT+few-shots prompting:<br>
- *customer:* "I want to change my subscription from standard to premium plan for next 3 months."

- Chatbot's Internal CoT:
  - Identify the user's intent: The user wants to upgrade their subscription.

  - Access user's account details: Retrieve the current plan ("Standard") and billing history.

  - Access subscription plan details: Find the cost of the "Standard" plan $$($10/month)$$ and the "Premium" plan $$($20/month)$$

  - Calculate the pro-rated fee: The customer is in the middle of their billing cycle.
    - Current plan cost: $10/month.
    - New plan cost: $20/month.
    - Time remaining in cycle: 15 days (out of 30).
    - The price difference is $10/month.
    - Calculate the pro-rated fee as $$(New plan cost - Old plan cost) * (Days  remaining / Total days in billing cycle)$$ <br>In this case, $$( \$20 - \$10 ) \\times ( \\frac{15}{30} ) = \$5$$

  - Identify additional fees/terms: Check for any specific upgrade fees or policy changes. The policy states a one-time administrative fee of $5 for a plan change.<br>

  - Synthesize the final cost: The total cost will be the pro-rated difference plus the administrative fee:$$(\$5 + \$5)=\$10$$
  - Formulate a transparent and helpful response: Break down the cost calculation, explain each component, and confirm the next step.<br>

What do you think the response would be?



---

In [127]:
# Hands-on excercise using OpenAI API and GPT4.1
# This hands-on exercise provides a CoT example that enables AI model articulate it reasoning behind the generated response, improving transperancy.

### About this excercise:
</br>

|Category|Description|
|:--|:--|
|Task |Customer Servicing Application that answers customer inquiries step-by-step|
|Difficuly Level|Intermediate|
|Skills|Python|
</br>

In [128]:
#Import display and Markdown from IPython for formatted rendering of generated response
from IPython.display import Markdown, display

In [129]:
def printmd(string):
    display(Markdown(string))

Access setup - This function allows secure access to user-defined secrets stored in the Colab environment, such as API keys.

In [130]:
from google.colab import userdata

Import - interact with the OpenAI API, allows us to make requests to models like GPT-4.1

In [131]:
from openai import OpenAI

In [132]:
client = OpenAI(api_key=userdata.get('openai_api_key'))

**Attempt 1:** Add few-shots or examples to force the generic model to think-out-loud before generating a response (in CoT style).

In [133]:
message = "I want to change my subscription from standard plan to premium plan for 3 months"

In [134]:
inputs = [
        {"role": "system", "content": "You are a helpful customer service assistant. Use step-by-step approach to service the customer request with Greetings, Response"},
        {"role": "user", "content": "request: I want to continue my current subscription plan"},
        {"role": "assistant", "content": "Category: Continuation"},
        {"role": "assistant", "content": "Greetings: Most Certainly, I can help you with the extension of your current plan"},
        {"role": "assistant", "content": "Intent: The user wants to continue their subscription as is."},
        {"role": "assistant", "content": "Intent: The user wants to continue their subscription as is."},
        {"role": "assistant", "content": "account-details: Retrieve the current plan 'Standard', 'Premium' and billing history."},
        {"role": "assistant", "content": "plan-details: Find the cost of the 'Standard' plan $10/month and the 'Premium' plan $20/month"},
        {"role": "assistant", "content": "pro-rated-fee: The customer is continuing same plan, so calculation is not required."},
        {"role": "assistant", "content": "additional-fees-terms: Check for any specific upgrade fees or policy changes. The policy states no additional fees for same plan continuation."},
        {"role": "assistant", "content": "final-cost: The total cost will be 0 pro-rated difference plus the 0 administrative fee:"},
        {"role": "assistant", "content": "response: Break down the cost calculation, explain each component, and confirm the next step."},
        {"role": "assistant", "content": "response: The total cost is $10 + $0 = $10. Customer will not be charged anything extra."},
        {"role": "assistant", "content": "response: The total cost for each following month after extension will remian same."},
        {"role": "assistant", "content": "feedback: Check if the user has any feedback on current plan. Thank customer for extension."},

        {"role": "user", "content": "request: I want to change my subscription from premium to standard plan"},
        {"role": "assistant", "content": "Category: Downgrade"},
        {"role": "assistant", "content": "Confidence: High"},
        {"role": "assistant", "content": "Intent: The user wants to downgrade their subscription."},
        {"role": "assistant", "content": "Greetings: Sure, I can help you with the change to a different plan"},
        {"role": "assistant", "content": "account-details: Retrieve the current plan 'Premium' and billing history."},
        {"role": "assistant", "content": "plan-details: Find the cost of the 'Standard' plan $10/month and the 'Premium' plan $20/month"},
        {"role": "assistant", "content": "pro-rated-fee: The customer is in the middle of their billing cycle."},
        {"role": "assistant", "content": "additional-fees-terms: Check for any specific upgrade fees or policy changes. The policy states a one-time administrative fee of $5 for a plan change."},
        {"role": "assistant", "content": "final-cost: The total cost will be the pro-rated difference plus the administrative fee:"},
        {"role": "assistant", "content": "response: Break down the cost calculation, explain each component, and confirm the next step."},
        {"role": "assistant", "content": "response: The total cost is -$10 + $5 = -$5. Customer will be saving $5 on the next billing cycle."},
        {"role": "assistant", "content": "response: The total cost for each following month after upgrade will $10."},
        {"role": "assistant", "content": "feedback: Check if the user has any feedback on current plan. ask reason for downgrade."},

        {"role": "user", "content": "request: I want to change my subscription from standard to premium plan"},
        {"role": "assistant", "content": "Category: Upgrade"},
        {"role": "assistant", "content": "Confidence: High"},
        {"role": "assistant", "content": "Intent: The user wants to upgrade their subscription."},
        {"role": "assistant", "content": "Greetings: Excellent Choice! Let me help you with the upgrade to a premium plan"},
        {"role": "assistant", "content": "account-details: Retrieve the current Standard plan and billing history."},
        {"role": "assistant", "content": "plan-details: Find the cost of the 'Standard' plan $10/month and the 'Premium' plan $20/month"},
        {"role": "assistant", "content": "pro-rated-fee: The customer is in the middle of their billing cycle."},
        {"role": "assistant", "content": "additional-fees-terms: Check for any specific upgrade fees or policy changes. The policy states a one-time administrative fee of $5 for a plan change."},
        {"role": "assistant", "content": "final-cost: The total cost will be the pro-rated difference plus the administrative fee:"},
        {"role": "assistant", "content": "response: Break down the cost calculation, explain each component, and confirm the next step."},
        {"role": "assistant", "content": "response: The total cost is $10 + $5 = $15."},
        {"role": "assistant", "content": "response: The total cost for each following month after upgrade will $20."},
        {"role": "assistant", "content": "feedback: Provide details of upgrade as 'Ulimited access' and congratulate on upgrade. Thank customer."},
        {"role": "user", "content": f"Answer the following question: {message}"}
    ]

In [135]:
response_cot_fewshots = client.chat.completions.create(
    model='gpt-4.1',
    messages= inputs,
    max_tokens=2480, #cost efficiency, concise response
    temperature=0 #want the model to be consistent and factual. Low temperature for consistent classification.
)

Print the response generated by AI model for a given prompt

In [136]:
about_this_print='Following is how the generic model responds with CoT and added few-shots to prompt:'
printmd('<div style="background-color: lightblue; padding: 10px;">%s</div><br>' % about_this_print)

response_text = response_cot_fewshots.choices[0].message.content.strip()
formatted_text = "<br>".join(response_text.split('\n'))
printmd('<div style="background-color: lightblue; padding: 10px;">%s</div>' % formatted_text)

<div style="background-color: lightblue; padding: 10px;">Following is how the generic model responds with CoT and added few-shots to prompt:</div><br>

<div style="background-color: lightblue; padding: 10px;">Greetings: Thank you for your request! I’d be happy to help you upgrade your subscription from the Standard plan to the Premium plan for 3 months.<br><br>Step 1: Plan Details  <br>- Standard Plan: $10/month  <br>- Premium Plan: $20/month  <br><br>Step 2: Upgrade Process  <br>- You will be upgraded to the Premium plan for the next 3 months.<br>- After 3 months, you can choose to continue with Premium, switch back to Standard, or cancel.<br><br>Step 3: Cost Calculation  <br>- Premium Plan for 3 months: $20 x 3 = $60  <br>- If you are upgrading in the middle of your current billing cycle, a pro-rated charge may apply for the remaining days of this month.<br>- There may be a one-time administrative fee of $5 for the plan change (please confirm if this applies to your account).<br><br>Step 4: Next Steps  <br>- Please confirm if you’d like to proceed with the upgrade for 3 months.<br>- Once confirmed, I will process the change and provide you with the exact billing details.<br><br>Would you like to continue with the upgrade to Premium for 3 months? If you have any questions or need more information, please let me know!</div>

Build information so it can be sent to the AI model as part of prompt.

**Attempt 2:** Add examples like few-shots to an evolved reasoning model so the model learns from examples and splits the complex task into multiple steps in CoT style.

In [137]:
response_auto_cot = client.responses.create(
    model='o4-mini',
    input=inputs,
    reasoning={"effort": "high"},
    max_output_tokens=10000,
    stream=False
)

Print the response generated by AI model for a given prompt

In [138]:
about_this_print='Following is how the evolved reasoning model responds to a CoT + few-shots prompt:<br><br>'
printmd('<div style="background-color: #a8ee90; padding: 10px;">%s</div>' % about_this_print)
printmd('<div style="background-color: #a8ee90; padding: 10px;">%s</div>' % response_auto_cot.output_text)

<div style="background-color: #a8ee90; padding: 10px;">Following is how the evolved reasoning model responds to a CoT + few-shots prompt:<br><br></div>

<div style="background-color: #a8ee90; padding: 10px;">Greetings: Hello! I’d be happy to help you upgrade from the Standard plan to the Premium plan for a three-month period.

Response:
1. Account & Plans  
   – Your current plan: Standard at $10 / month  
   – Premium plan: $20 / month  
   – One-time administrative fee for plan changes: $5  

2. Cost Breakdown for 3-Month Premium Term  
   – 3 months × $20 / month = $60  
   – + $5 admin fee  
   – Total due: $65  

3. Scheduling Your Upgrade  
   You have two options:  
   a) Effective at the start of your next billing cycle  
      • You’ll pay $65 upfront for three full months of Premium service.  
   b) Effective immediately  
      • We’ll prorate your remaining days on Standard toward Premium (you’ll only pay the difference for the rest of this month), plus the $5 fee.  
      • Then you’ll be charged $20 on each of the next two full monthly renewals.  

Please let me know which option you’d like—upgrade now or at your next cycle—and I’ll process it right away!</div>

---

##Congratulations on running this fun exercise!



CoT combined with few-shots prompting enables the AI model to articulate reasoning, making it visible to the customer and helping build confidence, generating more deterministic and consistent responses to complex problems. Customer experience quality improves, and this technique extends easily across domains for complex tasks. You just proved it yourself!<br>





**Fun Fact:** Did you notice, we chose different models to show the difference in results?<br>



**Extra Credit:** Try variations with evolved reasoning models and observe different responses. Have you tried the new GPT-5?<br>



**Pro Tip:** The true power of CoT is better realized when combined with other techniques like *few-shot* prompting. Customer service applications using this technique have reported a remarkable 60-80% higher customer satisfaction with just efficient promoting techniques!<br><br>



If you are an AI Enthusiast, don't stop here - you could start with your first, simple chatbot application based on CoT+few-shots. Happy Prompt Engineering!<br>